# 30 – Apply Rules (Dynamic SQL)

In [ ]:
from pyspark.sql import functions as F
dbutils.widgets.text("CATALOG","<CATALOG_NAME>")
catalog = dbutils.widgets.get("CATALOG")
spark.sql(f"USE CATALOG {catalog}")
spark.sql("USE SCHEMA gold")

# Ensure control tables exist and seed (optional to run separately)
spark.sql(f"""
USE CATALOG {catalog};
USE SCHEMA control;
CREATE TABLE IF NOT EXISTS control.risk_rules
(rule_id STRING, name STRING, segment STRING, condition_sql STRING,
 impact_column STRING, impact_value STRING, priority INT, enabled BOOLEAN,
 effective_from DATE, effective_to DATE, owner STRING, notes STRING)
USING DELTA;
""")

f = spark.table("gold.features").alias("f")
r = spark.table(f"{catalog}.control.risk_rules").where("enabled = true AND current_date BETWEEN effective_from AND coalesce(effective_to, date'2999-12-31')")

matches = None
for row in r.collect():
    pred = row["condition_sql"]
    part = f.selectExpr(
        "loan_id",
        f"'{row['rule_id']}' as rule_id",
        f"'{row['impact_column']}' as impact_column",
        f"'{row['impact_value']}' as impact_value",
        f"{row['priority']} as priority"
    ).where(pred)
    matches = part if matches is None else matches.unionByName(part)

if matches is None:
    spark.sql("""CREATE OR REPLACE TABLE gold.risk_eval AS
                 SELECT f.*, array() as matched_rules, 'Medium' as risk_band, 0 as risk_points
                 FROM gold.features f""")
else:
    from pyspark.sql.window import Window
    w = Window.partitionBy("loan_id","impact_column").orderBy(F.desc("priority"))
    top = matches.withColumn("rn", F.row_number().over(w)).where("rn=1")
    scored = (f.join(top, on="loan_id", how="left")
                .groupBy(f.columns)
                .agg(F.collect_list("rule_id").alias("matched_rules"),
                     F.max(F.when(F.col("impact_column")=="risk_band", F.col("impact_value"))).alias("risk_band"),
                     F.sum(F.when(F.col("impact_column")=="risk_points",
                                  F.regexp_extract(F.col("impact_value"), "[-+]?\d+", 0).cast("int")).otherwise(0)).alias("risk_points")))
    (scored.fillna({"risk_band":"Medium","risk_points":0,"matched_rules":[]})
           .write.mode("overwrite").saveAsTable("gold.risk_eval"))
print("Refreshed gold.risk_eval")
